In [1]:
# Improting Geo Json
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017/')
db = client.GeoComps
a=db.GeoComps.find()
b=pd.DataFrame(a)
b["headquarters"].head()
#b["headquarters"]["coordinates"]
b.shape

(2939, 10)

In [2]:
b.drop(columns=['_id'], inplace=True)
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75


In [16]:
radio=2000
geopoint=b["headquarters"][0]
geopoint

{'type': 'Point', 'coordinates': [-118.393064, 34.090368]}

In [17]:
def findNear(geopoint,radio):
     return list(db.GeoComps.find({
        "headquarters": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio,
         }
       }
    }))
geopoint=b["headquarters"][0]
lst=findNear(geopoint,radio)
lst

[{'_id': ObjectId('5d2759fa6497a5d8bff314d3'),
  'name': 'Geni',
  'lat': 34.090368,
  'lng': -118.393064,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]},
  'number_of_employees': 18,
  'category_code': 'web',
  'founded_year': 2006,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31ba1'),
  'name': 'MyFreeImplants',
  'lat': 34.087825,
  'lng': -118.379493,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.379493, 34.087825]},
  'number_of_employees': 2,
  'category_code': 'network_hosting',
  'founded_year': 2005,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31b66'),
  'name': 'Meteor Games',
  'lat': 34.09316,
  'lng': -118.378335,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.378335, 34.09316]},
  'number_of_employees': 70,
  'category_code': 'games_video',
  'founded_year': 2007,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31d19'),
  'name': 'Book

In [18]:
def sum_employees(lst):
    suma=0
    for i in lst:
        suma+= i["number_of_employees"]
    return suma

n=sum_employees(lst)
n


97

In [19]:
sum_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=findNear(geopoint,radio)
    n=sum_employees(lost)
    sum_column.append(n)


In [20]:
b["Employees_Near"]=sum_column

In [21]:
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,97,0.000000
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,2183,0.000000
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869,0.461538
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35,180,0.000000
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012,0.176471


In [22]:
def new_old(lst):
    old=0
    new=0
    for i in lst:
        if i["age"]=='Old':
            old+=1
        else:
            new+=1
     
    if old>0:
        ratio= new/old
        return ratio
    else:
        return 1.5



In [23]:
ratio_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=list(findNear(geopoint,radio))
    n=new_old(lost)
    ratio_column.append(n)


In [24]:
b["New/Old_Ratio"]=ratio_column

In [25]:
b.head(300)

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,97,0.000000
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,2183,0.000000
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869,0.272727
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35,180,0.000000
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012,0.393939
5,Old,games_video,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,272,0.111111
6,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.0899512...",37.423390,-122.089951,Jajah,110,1234,0.090909
7,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-74.3372, 40...",40.801358,-74.337200,AdaptiveBlue,15,15,0.000000
8,Old,web,USA,1995,"{'type': 'Point', 'coordinates': [-121.930035,...",37.295005,-121.930035,eBay,15000,15578,0.000000
9,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-122.07948, ...",37.392936,-122.079480,Ustream,250,2065,0.166667


TypeError: count() takes exactly one argument (0 given)